# 03 - Análisis Paramétrico Completo

Este notebook ejecuta el análisis paramétrico completo para todos los modelos generados.

## Objetivos:
- Analizar todos los modelos generados
- Ejecutar análisis estáticos y modales
- Generar resultados para reportes
- Monitorear el progreso

In [ ]:
import sys
sys.path.append('../src')

from analysis_engine import AnalysisEngine
import os
import time
from datetime import datetime

## 1. Inicializar el Motor de Análisis

In [ ]:
# Inicializar el motor de análisis
engine = AnalysisEngine(
    models_dir="../models",
    results_dir="../results"
)

print("Motor de análisis inicializado")
print(f"Directorio de modelos: {engine.models_dir}")
print(f"Directorio de resultados: {engine.results_dir}")

## 2. Obtener Lista de Modelos

In [ ]:
# Obtener todos los modelos disponibles
model_files = engine.get_model_files()

print(f"Total de modelos a analizar: {len(model_files)}")

if len(model_files) > 10:
    print("\nPrimeros 10 modelos:")
    for i, file in enumerate(model_files[:10]):
        print(f"{i+1}. {os.path.basename(file)}")
    print(f"... y {len(model_files) - 10} más")
else:
    print("\nTodos los modelos:")
    for i, file in enumerate(model_files):
        print(f"{i+1}. {os.path.basename(file)}")

## 3. Verificar Modelos Ya Analizados

In [ ]:
# Verificar qué modelos ya han sido analizados
existing_results = []
if os.path.exists(engine.results_dir):
    existing_results = [f for f in os.listdir(engine.results_dir) if f.endswith('_results.json')]

print(f"Modelos ya analizados: {len(existing_results)}")

# Filtrar modelos que aún no han sido analizados
model_names = [os.path.splitext(os.path.basename(f))[0] for f in model_files]
analyzed_names = [f.replace('_results.json', '') for f in existing_results]

pending_models = []
for model_file in model_files:
    model_name = os.path.splitext(os.path.basename(model_file))[0]
    if model_name not in analyzed_names:
        pending_models.append(model_file)

print(f"Modelos pendientes de análisis: {len(pending_models)}")

if len(pending_models) == 0:
    print("\n✅ Todos los modelos ya han sido analizados!")
else:
    print("\nModelos pendientes:")
    for i, file in enumerate(pending_models[:5]):
        print(f"{i+1}. {os.path.basename(file)}")
    if len(pending_models) > 5:
        print(f"... y {len(pending_models) - 5} más")

## 4. Ejecutar Análisis Paramétrico

In [ ]:
if pending_models:
    print("Iniciando análisis paramétrico...")
    print(f"Hora de inicio: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Ejecutar análisis para todos los modelos pendientes
    start_time = time.time()
    
    results = engine.analyze_multiple_models(pending_models)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print(f"\nAnálisis completado en {elapsed_time:.2f} segundos")
    print(f"Modelos analizados exitosamente: {len(results)}")
    
    # Estadísticas de éxito
    static_success = sum(1 for r in results if r['static_analysis']['success'])
    modal_success = sum(1 for r in results if r['modal_analysis']['success'])
    
    print(f"Análisis estáticos exitosos: {static_success}/{len(results)}")
    print(f"Análisis modales exitosos: {modal_success}/{len(results)}")
else:
    print("No hay modelos pendientes de análisis.")
    results = []

## 5. Cargar Todos los Resultados

In [ ]:
# Cargar todos los resultados disponibles
all_results = engine.load_all_results()

print(f"Total de resultados cargados: {len(all_results)}")

# Estadísticas generales
if all_results:
    static_success_total = sum(1 for r in all_results if r['static_analysis']['success'])
    modal_success_total = sum(1 for r in all_results if r['modal_analysis']['success'])
    
    print(f"\n=== ESTADÍSTICAS GENERALES ===")
    print(f"Total de modelos: {len(all_results)}")
    print(f"Análisis estáticos exitosos: {static_success_total}/{len(all_results)} ({static_success_total/len(all_results)*100:.1f}%)")
    print(f"Análisis modales exitosos: {modal_success_total}/{len(all_results)} ({modal_success_total/len(all_results)*100:.1f}%)")
    
    # Desplazamientos máximos
    max_displacements = [r['static_analysis']['max_displacement'] for r in all_results if r['static_analysis']['success']]
    if max_displacements:
        print(f"\nDesplazamientos máximos:")
        print(f"  Máximo: {max(max_displacements):.6f} m")
        print(f"  Mínimo: {min(max_displacements):.6f} m")
        print(f"  Promedio: {sum(max_displacements)/len(max_displacements):.6f} m")
    
    # Períodos fundamentales
    fundamental_periods = [r['modal_analysis']['periods'][0] for r in all_results 
                          if r['modal_analysis']['success'] and r['modal_analysis']['periods']]
    if fundamental_periods:
        print(f"\nPeríodos fundamentales:")
        print(f"  Máximo: {max(fundamental_periods):.4f} s")
        print(f"  Mínimo: {min(fundamental_periods):.4f} s")
        print(f"  Promedio: {sum(fundamental_periods)/len(fundamental_periods):.4f} s")

## 6. Verificar Archivos de Resultados

In [ ]:
# Verificar archivos generados
if os.path.exists(engine.results_dir):
    result_files = os.listdir(engine.results_dir)
    
    print("=== ARCHIVOS DE RESULTADOS ===")
    print(f"Total de archivos: {len(result_files)}")
    
    # Categorizar archivos
    json_files = [f for f in result_files if f.endswith('.json')]
    html_files = [f for f in result_files if f.endswith('.html')]
    
    print(f"Archivos JSON (resultados): {len(json_files)}")
    print(f"Archivos HTML (visualizaciones): {len(html_files)}")
    
    print("\nEjemplos de archivos JSON:")
    for i, file in enumerate(json_files[:5]):
        print(f"  {i+1}. {file}")
    
    if len(json_files) > 5:
        print(f"  ... y {len(json_files) - 5} más")

## 7. Resumen Final

In [ ]:
print("=== RESUMEN FINAL ===")
print(f"✓ Modelos totales: {len(model_files)}")
print(f"✓ Resultados generados: {len(all_results)}")
print(f"✓ Archivos de resultados: {len(json_files) if 'json_files' in locals() else 0}")
print(f"✓ Archivos de visualización: {len(html_files) if 'html_files' in locals() else 0}")

if 'elapsed_time' in locals():
    print(f"✓ Tiempo total de análisis: {elapsed_time:.2f} segundos")
    if len(pending_models) > 0:
        avg_time = elapsed_time / len(pending_models)
        print(f"✓ Tiempo promedio por modelo: {avg_time:.2f} segundos")

print("\n✅ Análisis paramétrico completado exitosamente!")
print("\nPróximos pasos:")
print("1. Ejecutar el notebook 04_generar_reportes.ipynb para crear reportes")
print("2. Revisar los archivos HTML de visualización")
print("3. Analizar los archivos CSV con los resultados")